In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT","DEFAULT_PROJECT_NAME")

In [6]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/langgraph_cloud")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/langgraph_cloud', 'title': 'LangGraph Platform | 🦜️🛠️ LangSmith', 'description': 'LangGraph Platform is a managed service for deploying and hosting LangGraph applications. Deploying applications with LangGraph Cloud shortens the time-to-market for developers. With one click, deploy a production-ready API with built-in persistence for your LangGraph application. LangGraph Cloud APIs are horizontally scalable and deployed with durable storage.', 'language': 'en'}, page_content='\n\n\n\n\nLangGraph Platform | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset 

load data -> docs -> divide into text into chunks -> text -> vector -> vector embedding -> vector storedb

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/langgraph_cloud', 'title': 'LangGraph Platform | 🦜️🛠️ LangSmith', 'description': 'LangGraph Platform is a managed service for deploying and hosting LangGraph applications. Deploying applications with LangGraph Cloud shortens the time-to-market for developers. With one click, deploy a production-ready API with built-in persistence for your LangGraph application. LangGraph Cloud APIs are horizontally scalable and deployed with durable storage.', 'language': 'en'}, page_content='LangGraph Platform | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/langgraph_cloud', 'title': 'LangGraph Platform | 🦜️🛠️ LangSmith', 'description': 'LangGraph Platform is a managed service for deploying and hosting LangGraph applications. Deploying applications with LangGraph Cloud shortens the time-to-market for developers. With one click, deploy a production-ready API with built-in persistence for your LangGraph a

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstoredb = FAISS.from_documents(documents, embeddings)

C:\Users\IT Tech\AppData\Local\Temp\ipykernel_13564\3155679792.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [16]:
vectorstoredb


In [10]:
query="LangGraph Platform is a managed service for deploying and hosting LangGraph applications."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangGraph Platform is a managed service for deploying and hosting LangGraph applications. Deploying applications with LangGraph Cloud shortens the time-to-market for developers. With one click, deploy a production-ready API with built-in persistence for your LangGraph application. LangGraph Cloud APIs are horizontally scalable and deployed with durable storage.'

In [14]:
from langchain_openai import OpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define an actual LLM (e.g., OpenAI)
llm = OpenAI(model="gpt-3.5-turbo")

# Define the Prompt
prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>    
    
    """
)

# Fix: Pass an actual LLM instead of an embedding model
document_chain = create_stuff_documents_chain(llm, prompt)

print(document_chain)


bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>    \n    \n    '), additional_kwargs={})])
| OpenAI(client=<openai.resources.completions.Completions object at 0x00000255257DAC00>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000025523393EF0>, model_name='gpt-3.5-turbo', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser() kwargs={} config={'run_name': 'stuff_documents_chain'} config_factories=[]


In [15]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangGraph Platform is a managed service for deploying and hosting LangGraph applications.",
    "context":[Document(page_content="LangGraph Platform is a managed service for deploying and hosting LangGraph applications. Deploying applications with LangGraph Cloud shortens the time-to-market for developers. With one click, deploy a production-ready API with built-in persistence for your LangGraph application. LangGraph Cloud APIs are horizontally scalable and deployed with durable storage.")]
})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [16]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [17]:
response=retrieval_chain.invoke({"input":"LangGraph Platform is a managed service for deploying and hosting LangGraph applications."})
response['answer']

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}